In [1]:
%matplotlib inline

import json

from pathlib import Path
import os

import matplotlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append(os.path.join(os.getcwd(), os.pardir, 'src'))

%load_ext autoreload

%autoreload 2
from visualization.visualize import *

pd.set_option('display.max_rows', 101)

In [29]:
from math import pi
import pandas as pd

from bokeh.io import output_notebook, show, save
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LinearColorMapper,
    BasicTicker,
    PrintfTickFormatter,
    ColorBar,
)
from bokeh.plotting import figure
from bokeh.sampledata.unemployment1948 import data


In [4]:
output_notebook()

Loading BokehJS ...

### Load metrics

In [5]:
who_df = pd.read_csv(Path('../data/interim/calc_cols_added.csv'), index_col=0)
code_to_name = (who_df[['Iso Code', 'Country Name']].drop_duplicates()
                                                    .set_index('Iso Code')
                                                    .to_dict()['Country Name'])

In [6]:
all_series = []
for f in Path("../data/processed/scores/").glob('*.csv'):
    print(f)
    all_series.append(pd.read_csv(f, index_col=0))
    
all_scores = pd.concat(all_series, axis=1)
all_scores.head(2)

../data/processed/scores/country-nasa_pct_diff_of_agg_births.csv
../data/processed/scores/country-wb-abs-crude_birth_rate.csv
../data/processed/scores/country-internal-dtp-dropout-agg.csv
../data/processed/scores/country-internal-dtp-dropout-averaged.csv
../data/processed/scores/country-internal-coef_of_var.csv
../data/processed/scores/country-weunic_diff_national.csv
../data/processed/scores/country-wb-abs-infant_mortality.csv


,nasa_pct_diff_of_agg,crude_birth_rate_wb_diff,dropout_aggregated,dropout_averaged,coef_of_var,wuenic_mean_difference_from_national,infant_mortality_wb_diff
AFG,104.205922,4.860682,11.649611,14.536052,0.065588,15.333333,18.287177
AGO,1.952697,10.140349,12.408592,18.459532,0.213387,10.545455,3.053068


In [7]:
pd.Series([5, 1, 3, np.nan, 4, np.nan, np.nan, 10]).rank()

0    4.0
1    1.0
2    2.0
3    NaN
4    3.0
5    NaN
6    NaN
7    5.0
dtype: float64

In [8]:
ranks_and_total = (
    all_scores.rank(ascending=False, method='dense')
              .apply(lambda x: x + pd.isnull(x).sum(), axis=0)
              .assign(total=lambda x: (x.mean(axis=1) * (pd.notnull(x).sum(axis=1))).rank())
)

In [15]:
for c in ranks_and_total.columns.tolist():
    print(f"'{c}': 'asdf',")

'nasa_pct_diff_of_agg': 'asdf',
'crude_birth_rate_wb_diff': 'asdf',
'dropout_aggregated': 'asdf',
'dropout_averaged': 'asdf',
'coef_of_var': 'asdf',
'wuenic_mean_difference_from_national': 'asdf',
'infant_mortality_wb_diff': 'asdf',
'total': 'asdf',


In [16]:
cols_remap = {
    'nasa_pct_diff_of_agg': 'NASA GPW: Births Percent Difference',
    'crude_birth_rate_wb_diff': 'Denominator: Compare to Crude Birth Rate',
    'dropout_aggregated': 'Coverage: Country-wide DTP Dropout',
    'dropout_averaged': 'Coverage: Average DTP Dropout per Region',
    'coef_of_var': 'Coverage: Variance in Number of Doses for Co-Admin Vax',
    'wuenic_mean_difference_from_national': 'WUENIC: Mean difference from WUENIC national',
    'infant_mortality_wb_diff': 'Denominator: Compare to Infant Mortality Rate',
    'total': 'Overall Score',   
}

In [25]:
col_order = [
    'Coverage: Average DTP Dropout per Region',
 'Coverage: Country-wide DTP Dropout',
 'Coverage: Variance in Number of Doses for Co-Admin Vax',
 'Denominator: Compare to Crude Birth Rate',
 'Denominator: Compare to Infant Mortality Rate',
 'NASA GPW: Births Percent Difference',
 'WUENIC: Mean difference from WUENIC national',

    'Overall Score',
]

In [31]:
rank_df = (pd.melt(ranks_and_total.reset_index(),
             id_vars=['index'],
             value_name='rate',
             var_name='measure')
           .replace({np.inf: np.nan, -np.inf: np.nan})
           .rename(columns={'index': 'ISO_Code'})
           .replace({'ISO_Code': code_to_name})
           .replace({'measure': cols_remap})
          )


# this is the colormap from the original NYTimes plot
colors = [matplotlib.colors.to_hex(c) for c in plt.cm.RdYlGn(np.linspace(0, 1, 100))]
mapper = LinearColorMapper(palette=colors,
                           low=rank_df.rate.min(),
                           high=rank_df.rate.max())

source = ColumnDataSource(rank_df)

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

ordered_countries = (ranks_and_total.sort_values('total', ascending=True, na_position='first')
                                    .rename(index=code_to_name)
                                    .index.values.tolist())


p = figure(title="",
           x_range=col_order,
           y_range=ordered_countries,
           x_axis_location="above",
           plot_width=600,
           plot_height=1500,
           tools=TOOLS, toolbar_location='below')

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "8pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = pi / 2

p.rect(x="measure",
       y="ISO_Code",
       width=1,
       height=1,
       source=source,
       fill_color={'field': 'rate', 'transform': mapper},
       line_color=None)


p.select_one(HoverTool).tooltips = [
     ('Country', '@ISO_Code'),
     ('rate', '@rate'),
]

save(p, filename="heatmap.html")
show(p)      # show the plot


/Users/bull/anaconda3/envs/subnational-vaccines/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/Users/bull/anaconda3/envs/subnational-vaccines/lib/python3.6/site-packages/bokeh/io/saving.py:138: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")
